In [6]:
from smolagents import InferenceClientModel, tool, CodeAgent,  load_tool, DuckDuckGoSearchTool, FinalAnswerTool
import pytz
import datetime
import requests

In [17]:
system_prompt = """You are an intelligent AI assistant that can use tools to solve complex tasks.

THINKING PROCESS:
1. Always break down complex requests into logical steps
2. Use the most appropriate tool for each sub-task
3. Combine information from multiple tools when needed
4. Be creative and contextual in your reasoning

TOOL USAGE GUIDELINES:
- For time-related questions: Use get_time with specific IANA timezones (e.g., "Europe/London", not just "London")
- For image generation: Create detailed, contextual prompts that incorporate other information you've gathered
- For information searches: Use DuckDuckGoSearchTool when you need current facts or additional context

CREATIVE PROMPT ENGINEERING:
- When generating images, incorporate specific details from other tools (like exact times, locations, or facts)
- Build dynamic prompts that tell a story or create meaningful connections
- Avoid generic or cliché image prompts

EXAMPLE WORKFLOW:
User: "What time in Tokyo and generate something related"
→ Step 1: get_time("Asia/Tokyo") 
→ Step 2: image_gen("A futuristic Tokyo cityscape at [exact time from step 1] with neon lights reflecting the time")

Always think step by step and create meaningful connections between tools!"""

In [7]:
@tool
def get_time(timezone : str) -> str:
    """
    A tool that fetches the current local time of a given timezone.
    Args:
    timezone: A string reprsenting the valid timezone. Eg; (India/America)
    """
    try:
        # Create timezone object
        tz = pytz.timezone(timezone)
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"

In [13]:
@tool
def image_gen(prompt : str) -> str:
    """
    A tool to generate an image based on the prompt.
    Args:
    prompt: A string to explain what image is to be generated. Eg: (Funny cat image/London Bus)
    """
    try:
        from huggingface_hub import InferenceClient
        client = InferenceClient(token="hf_token")  
        image = client.text_to_image(prompt, model="black-forest-labs/FLUX.1-schnell")
        image.save("my_image.png")
        return f"✅ Image generated and saved for: '{prompt}'"
    except Exception as e:
        return f"❌ Image generation failed: {str(e)}"

In [9]:
final_answer = FinalAnswerTool()

In [1]:
model = InferenceClientModel(
    token = "hf_token",
    max_tokens=2096,
    temperature=0.5,
    model_id='meta-llama/Llama-4-Scout-17B-16E-Instruct'
)

NameError: name 'InferenceClientModel' is not defined

In [22]:
Agent = CodeAgent(
    model = model,
    tools = [final_answer, image_gen, get_time, DuckDuckGoSearchTool()],
    max_steps=6,
    verbosity_level=1,
    instructions=system_prompt
)  

In [24]:
Agent.run("what is the current time in london and generate an image regarding that subject.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ what is the current time in london and generate an image regarding that subject.                                │
│                                                                                                                 │
╰─ InferenceClientModel - meta-llama/Llama-4-Scout-17B-16E-Instruct ──────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Get the current time in London                                                                         
  current_time_london = get_time("Europe/London")                                                                  
  print(current_time_london)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The current local time in Europe/London is: 2025-11-03 15:29:52

Out: None

[Step 1: Duration 2.55 seconds| Input tokens: 2,379 | Output tokens: 77]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The current time in London is 2025-11-03 15:29:52. An image of a scene in London at 3:29 PM on a   
  sunny afternoon in November with people walking in the streets has been generated.")                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: The current time in London is 2025-11-03 15:29:52. An image of a scene in London at 3:29 PM on a 
sunny afternoon in November with people walking in the streets has been generated.

[Step 3: Duration 1.15 seconds| Input tokens: 7,735 | Output tokens: 339]

'The current time in London is 2025-11-03 15:29:52. An image of a scene in London at 3:29 PM on a sunny afternoon in November with people walking in the streets has been generated.'